In [25]:
import os
import re
import json
import requests 
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
if not os.path.isdir('data/'):
    os.mkdir('data/')

# Understat.com

Followed this Kaggle [tutorial](https://www.kaggle.com/slehkyi/web-scraping-football-statistics-2014-now#Scraping-data-for-all-teams-of-all-leagues-of-all-seasons).

In [3]:
data_path = 'data/understat/'

if not os.path.isdir(data_path):
    os.mkdir(data_path)

In [4]:
# create urls for all seasons of all leagues
base_url = 'https://understat.com/league'
leagues = ['La_liga', 'EPL', 'Bundesliga', 'Serie_A', 'Ligue_1']
seasons = ['2014', '2015', '2016', '2017', '2018', '2019', '2020']

In [17]:
full_data = dict()
for league in leagues:
  
    season_data = dict()
    for season in seasons:    
        url = base_url+'/'+league+'/'+season
        res = requests.get(url)
        soup = BeautifulSoup(res.content, "html.parser")

        # Based on the structure of the webpage, I found that data is in the JSON variable, under <script> tags
        scripts = soup.find_all('script')

        string_with_json_obj = ''

        # Find data for teams
        for el in scripts:
            if 'teamsData' in str(el):
                string_with_json_obj = str(el).strip()

        # print(string_with_json_obj)

        # strip unnecessary symbols and get only JSON data
        ind_start = string_with_json_obj.index("('")+2
        ind_end = string_with_json_obj.index("')")
        json_data = string_with_json_obj[ind_start:ind_end]
        json_data = json_data.encode('utf8').decode('unicode_escape')


        # convert JSON data into Python dictionary
        data = json.loads(json_data)

        # Get teams and their relevant ids and put them into separate dictionary
        teams = {}
        for id in data.keys():
            teams[id] = data[id]['title']

        # EDA to get a feeling of how the JSON is structured
        # Column names are all the same, so we just use first element
        columns = []
        # Check the sample of values per each column
        values = []
        for id in data.keys():
            columns = list(data[id]['history'][0].keys())
            values = list(data[id]['history'][0].values())
            break

        # Getting data for all teams
        dataframes = {}
        for id, team in teams.items():
            teams_data = []
            for row in data[id]['history']:
                teams_data.append(list(row.values()))

            df = pd.DataFrame(teams_data, columns=columns)
            dataframes[team] = df

        for team, df in dataframes.items():
            dataframes[team]['ppda_coef'] = \
                dataframes[team]['ppda'].apply(lambda x: x['att']/x['def'] if x['def'] != 0 else 0)
            dataframes[team]['oppda_coef'] = \
                dataframes[team]['ppda_allowed'].apply(lambda x: x['att']/x['def'] if x['def'] != 0 else 0)

        cols_to_sum = ['xG', 'xGA', 'npxG', 'npxGA', 'deep', 'deep_allowed', 'scored', 'missed', 'xpts', 
                       'wins', 'draws', 'loses', 'pts', 'npxGD']
        cols_to_mean = ['ppda_coef', 'oppda_coef']

        frames = []
        for team, df in dataframes.items():
            sum_data = pd.DataFrame(df[cols_to_sum].sum()).transpose()
            mean_data = pd.DataFrame(df[cols_to_mean].mean()).transpose()
            final_df = sum_data.join(mean_data)
            final_df['team'] = team
            final_df['matches'] = len(df)
            frames.append(final_df)

        full_stat = pd.concat(frames)

        team_attrs = ['team', 'matches', 'wins', 'draws', 'loses', 'scored', 'missed', 'pts', 'xG',
                      'npxG', 'xGA', 'npxGA', 'npxGD', 'ppda_coef', 'oppda_coef', 'deep', 'deep_allowed', 'xpts']
        full_stat = full_stat[team_attrs]
        full_stat.sort_values('pts', ascending=False, inplace=True)
        full_stat.reset_index(inplace=True, drop=True)
        full_stat['position'] = range(1,len(full_stat)+1)

        full_stat['xG_diff'] = full_stat['xG'] - full_stat['scored']
        full_stat['xGA_diff'] = full_stat['xGA'] - full_stat['missed']
        full_stat['xpts_diff'] = full_stat['xpts'] - full_stat['pts']

        cols_to_int = ['wins', 'draws', 'loses', 'scored', 'missed', 'pts', 'deep', 'deep_allowed']
        full_stat[cols_to_int] = full_stat[cols_to_int].astype(int)

        col_order = ['position', 'team', 'matches', 'wins', 'draws', 'loses', 'scored', 'missed', 
                     'pts', 'xG', 'xG_diff', 'npxG', 'xGA', 'xGA_diff', 'npxGA', 'npxGD', 'ppda_coef', 
                     'oppda_coef', 'deep', 'deep_allowed', 'xpts', 'xpts_diff']
        full_stat = full_stat[col_order]
        full_stat = full_stat.set_index('position')
        # print(full_stat.head(20))

        season_data[season] = full_stat

    df_season = pd.concat(season_data)
    full_data[league] = df_season

data = pd.concat(full_data)
data.head()

team  matches  wins  draws  loses  scored  \
             position                                                         
La_liga 2014 1               Barcelona       38    30      4      4     110   
             2             Real Madrid       38    30      2      6     118   
             3         Atletico Madrid       38    23      9      6      67   
             4                Valencia       38    22     11      5      70   
             5                 Sevilla       38    23      7      8      71   

                       missed  pts          xG    xG_diff  ...        xGA  \
             position                                      ...              
La_liga 2014 1             21   94  102.980152  -7.019848  ...  28.444293   
             2             38   92   95.766243 -22.233757  ...  42.607198   
             3             29   78   57.047670  -9.952330  ...  29.069107   
             4             32   77   55.062500 -14.937500  ...  39.392572   
             5             45   76   69.526624  -1.473376  ...  47.862742   

                       xGA_diff      npxGA      npxGD  ppda_coef  oppda_coef  \
             position                                                          
La_liga 2014 1         7.444293  24.727907  73.049305   5.683535   16.367593   
             2         4.607198  38.890805  47.213090  10.209085   12.929510   
             3         0.069107  26.839271  25.748737   8.982028    9.237091   
             4         7.392572  33.446477  16.257501   8.709827    7.870225   
             5         2.862742  41.916529  20.178070   8.276148    9.477805   

                       deep  deep_allowed     xpts  xpts_diff  
             position                                          
La_liga 2014 1          489           114  94.0813     0.0813  
             2          351           153  81.7489   -10.2511  
             3          197           123  73.1353    -4.8647  
             4          203           172  63.7068   -13.2932  
             5          305           168  67.3867    -8.6133  

[5 rows x 21 columns]

In [18]:
data.to_csv(data_path + 'understat.csv')

# Fbref.com

Inspired from this [Github](https://github.com/parth1902/Scrape-FBref-data/).

In [44]:
with open('attributes.json', 'r', encoding='utf-8') as f:
    attributes = json.load(f)
    
# standard stats)
stats = attributes['stats']
stats2 = attributes['stats2']

# goalkeeping
keepers = attributes['keepers']
keepers2 = attributes['keepers2']

# advanced goalkeeping 
keepersadv = attributes['keepersadv']
keepersadv2 = attributes['keepersadv2']

# shooting
shooting = attributes['shooting']
shooting2 = attributes['shooting2']
shooting3 = attributes['shooting3']

# passing
passing = attributes['passing']
passing2 = attributes['passing2']

# passtypes
passing_types = attributes['passing_types']
passing_types2 = attributes['passing_types2']

# goal and shot creation
gca = attributes['gca']
gca2 = attributes['gca2']

# defensive actions
defense = attributes['defense']
defense2 = attributes['defense2']

# possession
possession = attributes['possession']
possession2 = attributes['possession2']

# playingtime
playingtime = attributes['playingtime']
playingtime2 = attributes['playingtime2']

# miscallaneous
misc = attributes['misc']
misc2 = attributes['misc2']

In [76]:
#Functions to get the data in a dataframe using BeautifulSoup

def get_tables(url):
    res = requests.get(url)
    ## The next two lines get around the issue with comments breaking the parsing.
    comm = re.compile("<!--|-->")
    soup = BeautifulSoup(comm.sub("", res.text), 'lxml')
    all_tables = soup.findAll("tbody")
    
    team_table = all_tables[0]
    player_table = all_tables[1]
    
    return player_table, team_table


def get_frame(features, player_table):
    pre_df_player = dict()
    features_wanted_player = features
    rows_player = player_table.find_all('tr')
    
    for row in rows_player:
        if(row.find('th', {"scope":"row"}) != None):
    
            for f in features_wanted_player:
                cell = row.find("td", {"data-stat": f})
                a = cell.text.strip().encode()
                text = a.decode("utf-8")
                if(text == ''):
                    text = '0'
                
                if((f != 'player') & (f != 'nationality') & (f != 'position') & (f != 'squad') & (f != 'age') \
                   &(f != 'birth_year')):
                    text = float(text.replace(',',''))
                
                if f in pre_df_player:
                    pre_df_player[f].append(text)
                else:
                    pre_df_player[f] = [text]
                    
    df_player = pd.DataFrame.from_dict(pre_df_player)
    
    return df_player


def get_frame_team(features, team_table, big5):
    pre_df_squad = dict()
    #Note: features does not contain squad name, it requires special treatment
    features_wanted_squad = features
    rows_squad = team_table.find_all('tr')
    
    squad_tag = 'td' if big5 else 'th'
    
    for row in rows_squad:
        if(row.find('th', {"scope":"row"}) != None and row.find(squad_tag,{"data-stat":"squad"}) != None):
            name = row.find(squad_tag,{"data-stat":"squad"}).text.strip().encode().decode("utf-8")
            if 'squad' in pre_df_squad:
                pre_df_squad['squad'].append(name)
            else:
                pre_df_squad['squad'] = [name]
            
            if big5:
                league = row.find('td', {"data-stat":"comp"}).text.strip().encode().decode("utf-8")
                pre_df_squad['league'] = league
            
            for f in features_wanted_squad:
                cell = row.find("td", {"data-stat": f})
                a = cell.text.strip().encode()
                text = a.decode("utf-8")
                if(text == ''):
                    text = '0'
                if((f!='player') & (f!='nationality') & (f!='position') & (f!='squad') & (f!='age') \
                   & (f!='birth_year')):
                    text = float(text.replace(',',''))
                if f in pre_df_squad:
                    pre_df_squad[f].append(text)
                else:
                    pre_df_squad[f] = [text]
    
    df_squad = pd.DataFrame.from_dict(pre_df_squad)
    
    return df_squad


def frame_for_category(category, top, end, features):
    url = (top + category + end)
    player_table, team_table = get_tables(url)
    df_player = get_frame(features, player_table)
    
    return df_player


def frame_for_category_team(category, top, end, features, big5):
    url = (top + category + end)
    player_table, team_table = get_tables(url)
    if big5:
        df_team = get_frame_team(features, player_table, big5)
    else:
        df_team = get_frame_team(features, team_table, big5)
    
    return df_team


#Function to get the player data for outfield player, includes all categories - standard stats, shooting
#passing, passing types, goal and shot creation, defensive actions, possession, and miscallaneous
def get_outfield_data(top, end):
    attrs = [stats, shooting2, passing2, passing_types2, gca2, defense2, possession2, misc2]
    attrs_names = ['stats', 'shooting', 'passing', 'passing_types', 'gca', 'defense', 'possession', 'misc']
    
    dfs = []
    for i, attr in enumerate(attrs_names):
        dfs.append(frame_for_category(attr, top, end, attrs[i]))
    
    df = pd.concat(dfs, axis=1)
    df = df.loc[:,~df.columns.duplicated()]
    
    return df


#Function to get keeping and advance goalkeeping data
def get_keeper_data(top, end):
    df1 = frame_for_category('keepers', top, end,keepers)
    df2 = frame_for_category('keepersadv', top, end,keepersadv2)
    df = pd.concat([df1, df2], axis=1)
    df = df.loc[:,~df.columns.duplicated()]
    
    return df


#Function to get team-wise data accross all categories as mentioned above
def get_team_data(top, end, big5=False):
    attrs = [stats2, keepers2, keepersadv2, shooting3, passing2, passing_types2, gca2, defense2, possession2, misc2]
    attrs_names = ['stats', 'keepers', 'keepersadv', 'shooting', 'passing', 'passing_types', 'gca', 'defense', 
                  'possession', 'misc']
    
    dfs = []
    for i, attr in enumerate(attrs_names):
        dfs.append(frame_for_category_team(attr, top, end, attrs[i], big5))
    
    df = pd.concat(dfs, axis=1)
    df = df.loc[:,~df.columns.duplicated()]
    
    return df

In [33]:
df_outfield = get_outfield_data('https://fbref.com/en/comps/9/','/Premier-League-Stats')
df_outfield

,player,nationality,position,squad,age,birth_year,games,games_starts,minutes,goals,...,fouls,fouled,offsides,pens_won,pens_conceded,own_goals,ball_recoveries,aerials_won,aerials_lost,aerials_won_pct
0,Patrick van Aanholt,nl NED,DF,Crystal Palace,30-122,1990,10.0,9.0,829.0,0.0,...,6.0,3.0,3.0,0.0,0.0,0.0,96.0,6.0,7.0,46.2
1,Tammy Abraham,eng ENG,FW,Chelsea,23-088,1997,14.0,8.0,782.0,6.0,...,14.0,10.0,3.0,1.0,0.0,0.0,39.0,32.0,22.0,59.3
2,Che Adams,eng ENG,FW,Southampton,24-169,1996,15.0,15.0,1289.0,4.0,...,15.0,23.0,10.0,0.0,0.0,0.0,58.0,19.0,72.0,20.9
3,Tosin Adarabioyo,eng ENG,DF,Fulham,23-096,1997,11.0,11.0,990.0,0.0,...,6.0,4.0,1.0,0.0,0.0,0.0,100.0,25.0,12.0,67.6
4,Adrián,es ESP,GK,Liverpool,33-361,1987,2.0,2.0,180.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,14.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
451,Andre-Frank Zambo Anguissa,cm CMR,MF,Fulham,25-043,1995,15.0,13.0,1194.0,0.0,...,22.0,12.0,0.0,0.0,0.0,0.0,149.0,18.0,14.0,56.3
452,Andi Zeqiri,ch SUI,DF,Brighton,21-190,1999,1.0,0.0,19.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,50.0
453,Oleksandr Zinchenko,ua UKR,DF,Manchester City,24-014,1996,1.0,0.0,13.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
454,Hakim Ziyech,ma MAR,FW,Chelsea,27-285,1993,7.0,5.0,388.0,1.0,...,6.0,8.0,0.0,0.0,0.0,0.0,40.0,0.0,6.0,0.0


In [46]:
df_team = get_team_data('https://fbref.com/en/comps/12/','/La-Liga-Stats')
df_team

,squad,players_used,possession,games,games_starts,minutes,goals,assists,pens_made,pens_att,...,fouls,fouled,offsides,pens_won,pens_conceded,own_goals,ball_recoveries,aerials_won,aerials_lost,aerials_won_pct
0,vs Alavés,25.0,61.0,15.0,165.0,1350.0,17.0,10.0,1.0,1.0,...,187.0,193.0,54.0,1.0,2.0,0.0,1483.0,231.0,367.0,38.6
1,vs Athletic Club,24.0,47.4,15.0,165.0,1350.0,17.0,11.0,3.0,3.0,...,222.0,193.0,41.0,3.0,2.0,1.0,1305.0,282.0,332.0,45.9
2,vs Atlético Madrid,22.0,47.4,13.0,143.0,1170.0,4.0,3.0,0.0,0.0,...,156.0,167.0,11.0,0.0,4.0,1.0,1088.0,170.0,182.0,48.3
3,vs Barcelona,24.0,36.4,14.0,154.0,1260.0,14.0,6.0,2.0,2.0,...,200.0,120.0,24.0,2.0,3.0,3.0,990.0,124.0,151.0,45.1
4,vs Betis,25.0,42.5,15.0,165.0,1350.0,24.0,16.0,5.0,7.0,...,215.0,195.0,71.0,5.0,2.0,0.0,1285.0,282.0,256.0,52.4
5,vs Cádiz,28.0,65.6,15.0,165.0,1350.0,19.0,17.0,1.0,1.0,...,222.0,168.0,48.0,1.0,0.0,1.0,1465.0,308.0,262.0,54.0
6,vs Celta Vigo,22.0,48.1,15.0,165.0,1350.0,20.0,12.0,3.0,3.0,...,223.0,241.0,24.0,2.0,3.0,0.0,1414.0,269.0,243.0,52.5
7,vs Eibar,25.0,50.7,15.0,165.0,1350.0,15.0,13.0,0.0,1.0,...,185.0,209.0,58.0,1.0,4.0,0.0,1616.0,445.0,393.0,53.1
8,vs Elche,25.0,49.5,13.0,143.0,1170.0,16.0,14.0,2.0,4.0,...,184.0,162.0,20.0,4.0,1.0,0.0,1127.0,177.0,164.0,51.9
9,vs Getafe,22.0,56.0,14.0,154.0,1260.0,14.0,8.0,4.0,4.0,...,216.0,235.0,40.0,3.0,1.0,0.0,1429.0,376.0,314.0,54.5


In [75]:
df_team = get_team_data('https://fbref.com/en/comps/11/','/Serie-A-Stats')
df_team

,squad,players_used,possession,games,games_starts,minutes,goals,assists,pens_made,pens_att,...,fouls,fouled,offsides,pens_won,pens_conceded,own_goals,ball_recoveries,aerials_won,aerials_lost,aerials_won_pct
0,vs Atalanta,27.0,46.5,13.0,143.0,1170.0,20.0,18.0,1.0,3.0,...,180.0,195.0,12.0,2.0,2.0,0.0,1177.0,156.0,224.0,41.1
1,vs Benevento,21.0,57.6,14.0,154.0,1260.0,23.0,20.0,2.0,2.0,...,229.0,167.0,38.0,1.0,2.0,0.0,1162.0,226.0,202.0,52.8
2,vs Bologna,29.0,48.4,14.0,154.0,1260.0,25.0,18.0,2.0,2.0,...,191.0,188.0,29.0,2.0,1.0,2.0,1233.0,224.0,226.0,49.8
3,vs Cagliari,25.0,56.9,14.0,154.0,1260.0,27.0,20.0,2.0,2.0,...,190.0,181.0,36.0,2.0,2.0,0.0,1169.0,221.0,185.0,54.4
4,vs Crotone,27.0,51.6,14.0,154.0,1260.0,29.0,20.0,2.0,2.0,...,204.0,195.0,44.0,1.0,3.0,0.0,1249.0,197.0,152.0,56.4
5,vs Fiorentina,25.0,47.9,14.0,154.0,1260.0,20.0,12.0,3.0,4.0,...,221.0,198.0,34.0,4.0,2.0,1.0,1209.0,217.0,209.0,50.9
6,vs Genoa,29.0,55.2,14.0,154.0,1260.0,26.0,20.0,3.0,3.0,...,196.0,200.0,22.0,3.0,1.0,0.0,1421.0,239.0,197.0,54.8
7,vs Hellas Verona,27.0,53.5,14.0,154.0,1260.0,13.0,11.0,1.0,2.0,...,210.0,220.0,17.0,2.0,3.0,2.0,1563.0,339.0,314.0,51.9
8,vs Inter,22.0,45.6,14.0,154.0,1260.0,17.0,12.0,1.0,2.0,...,168.0,208.0,16.0,2.0,3.0,2.0,1017.0,176.0,210.0,45.6
9,vs Juventus,26.0,40.2,13.0,143.0,1170.0,12.0,7.0,2.0,2.0,...,179.0,156.0,17.0,1.0,5.0,0.0,950.0,132.0,172.0,43.4


In [50]:
df_keeper = get_keeper_data('https://fbref.com/en/comps/Big5/','/players/Big-5-European-Leagues-Stats')
df_keeper

,player,nationality,position,squad,age,birth_year,games_gk,games_starts_gk,minutes_gk,goals_against_gk,...,passes_length_avg_gk,goal_kicks,pct_goal_kicks_launched,goal_kick_length_avg,crosses_gk,crosses_stopped_gk,crosses_stopped_pct_gk,def_actions_outside_pen_area_gk,def_actions_outside_pen_area_per90_gk,avg_distance_def_actions_gk
0,Adrián,es ESP,GK,Liverpool,33-361,1987,2.0,2.0,180.0,9.0,...,36.0,14.0,50.0,38.7,18.0,2.0,11.1,0.0,0.00,12.3
1,Alisson,br BRA,GK,Liverpool,28-088,1992,12.0,12.0,1080.0,11.0,...,32.2,66.0,36.4,35.7,72.0,2.0,2.8,13.0,1.08,17.2
2,Saturnin Allagbé,bj BEN,GK,Dijon,27-037,1993,4.0,4.0,360.0,8.0,...,35.1,27.0,40.7,34.9,35.0,0.0,0.0,5.0,1.25,16.4
3,Nícolas Andrade,br BRA,GK,Udinese,32-261,1988,2.0,2.0,180.0,5.0,...,32.1,9.0,55.6,47.4,16.0,0.0,0.0,0.0,0.00,15.2
4,Alphonse Areola,fr FRA,GK,Fulham,27-306,1993,14.0,14.0,1260.0,21.0,...,36.7,95.0,66.3,47.9,130.0,19.0,14.6,8.0,0.57,13.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150,Iván Villar,es ESP,GK,Celta Vigo,23-173,1997,7.0,7.0,630.0,10.0,...,39.2,56.0,46.4,40.8,67.0,5.0,7.5,8.0,1.14,17.2
151,Rubén Yáñez,es ESP,GK,Getafe,27-078,1993,2.0,2.0,180.0,1.0,...,55.7,12.0,83.3,61.2,18.0,2.0,11.1,1.0,0.50,13.5
152,Robin Zentner,de GER,GK,Mainz 05,26-062,1994,13.0,13.0,1170.0,26.0,...,34.8,106.0,55.7,47.3,130.0,10.0,7.7,12.0,0.92,14.0
153,Ron-Robert Zieler,de GER,GK,Köln,31-321,1989,1.0,0.0,51.0,1.0,...,35.0,6.0,83.3,49.7,10.0,0.0,0.0,0.0,0.00,11.8


In [77]:
df_team_big5 = get_team_data('https://fbref.com/en/comps/Big5/','/squads/Big-5-European-Leagues-Stats', big5=True)
df_team_big5

,squad,league,players_used,possession,games,games_starts,minutes,goals,assists,pens_made,...,fouls,fouled,offsides,pens_won,pens_conceded,own_goals,ball_recoveries,aerials_won,aerials_lost,aerials_won_pct
0,vs Alavés,eng Premier League,25.0,61.0,15.0,165.0,1350.0,17.0,10.0,1.0,...,187.0,193.0,54.0,1.0,2.0,0.0,1483.0,231.0,367.0,38.6
1,vs Angers,eng Premier League,27.0,54.3,17.0,187.0,1530.0,26.0,20.0,2.0,...,241.0,196.0,18.0,2.0,2.0,0.0,1526.0,263.0,222.0,54.2
2,vs Arminia,eng Premier League,24.0,55.2,13.0,143.0,1170.0,23.0,18.0,2.0,...,184.0,158.0,31.0,3.0,0.0,1.0,1328.0,298.0,298.0,50.0
3,vs Arsenal,eng Premier League,25.0,48.1,15.0,165.0,1350.0,16.0,11.0,0.0,...,176.0,148.0,18.0,1.0,3.0,0.0,1260.0,238.0,186.0,56.1
4,vs Aston Villa,eng Premier League,19.0,50.3,14.0,154.0,1260.0,14.0,10.0,0.0,...,230.0,159.0,14.0,1.0,4.0,1.0,1250.0,270.0,330.0,45.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191,Werder Bremen,eng Premier League,25.0,43.8,13.0,143.0,1170.0,15.0,12.0,3.0,...,162.0,187.0,26.0,2.0,3.0,1.0,1183.0,221.0,266.0,45.4
192,West Brom,eng Premier League,25.0,39.8,15.0,165.0,1350.0,10.0,6.0,0.0,...,169.0,168.0,32.0,0.0,4.0,2.0,1374.0,267.0,289.0,48.0
193,West Ham,eng Premier League,21.0,41.5,15.0,165.0,1350.0,21.0,16.0,0.0,...,160.0,146.0,29.0,0.0,4.0,0.0,1344.0,305.0,287.0,51.5
194,Wolfsburg,eng Premier League,25.0,51.7,13.0,143.0,1170.0,20.0,16.0,1.0,...,167.0,143.0,30.0,1.0,3.0,1.0,1383.0,232.0,231.0,50.1
